In [ ]:
import os, sys, time
import numpy as np
from numpy import sqrt, exp, pi, square
import pandas as pd
pd.options.mode.chained_assignment = None        # default='warn'
import matplotlib
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'   # enable if you have a retina display
from scipy.optimize import curve_fit, minimize
from scipy.interpolate import interp1d, UnivariateSpline
import warnings
warnings.filterwarnings('ignore')
from multihist import Histdd, Hist1d
from tqdm import tqdm
from multiprocessing import Pool
from contextlib import contextmanager

In [ ]:
#### Plotting ####
params = {
    'backend': 'Agg',
    # colormap
    'image.cmap' : 'viridis',
    # figure
    'figure.figsize' : (4, 2),
    'font.size' : 32,
    'font.family' : 'serif',
    'font.serif' : ['Times'],
    # axes
    'axes.titlesize' : 42,
    'axes.labelsize' : 32,
    'axes.linewidth' : 2,
    # ticks
    'xtick.labelsize' : 24,
    'ytick.labelsize' : 24,
    'xtick.major.size' : 16,
    'xtick.minor.size' : 8,
    'ytick.major.size' : 16,
    'ytick.minor.size' : 8,
    'xtick.major.width' : 2,
    'xtick.minor.width' : 2,
    'ytick.major.width' : 2,
    'ytick.minor.width' : 2,
    'xtick.direction' : 'in',
    'ytick.direction' : 'in',
    # markers
    'lines.markersize' : 12,
    'lines.markeredgewidth' : 3,
    'errorbar.capsize' : 8,
    'lines.linewidth' : 3,
    #'lines.linestyle' : None,
    'lines.marker' : None,
    'savefig.bbox' : 'tight',
    'legend.fontsize' : 24,
    #'legend.fontsize': 18,
    #'figure.figsize': (15, 5),
    #'axes.labelsize': 18,
    #'axes.titlesize':18,
    #'xtick.labelsize':14,
    #'ytick.labelsize':14
    'axes.labelsize': 32,
    'axes.titlesize' : 32,
    'xtick.labelsize' : 25,
    'ytick.labelsize' : 25,
    'xtick.major.pad' : 10,
    'text.latex.unicode': True,
}
plt.rcParams.update(params)
plt.rc('text', usetex=False)

def plt_config(title=None, xlim=None, ylim=None, xlabel=None, ylabel=None, colorbar=False, sci=False):
    for field in ['title', 'xlim', 'ylim', 'xlabel', 'ylabel']:
        if eval(field) != None: getattr(plt, field)(eval(field))
    if isinstance(sci, str): plt.ticklabel_format(style='sci', axis=sci, scilimits=(0,0))
    if isinstance(colorbar,str): plt.colorbar(label=colorbar)
    elif colorbar: plt.colorbar(label = '$Number\ of\ Entries$')

from contextlib import contextmanager
@contextmanager
def initiate_plot(dimx=24, dimy=9):
    plt.rcParams['figure.figsize'] = (dimx, dimy)
    global fig; fig = plt.figure()
    yield
    plt.show()

In [ ]:
# Read real data
class read_n_concat():   
    __version__ = '0.0.3'

    def __init__(self):
        self.pre_selection = ['True',]
        self.pre_selection = '&'.join(self.pre_selection)
        self._read_flag = True
    
    def process(self, indir = '', number_file_cap = 0, number_cup = 1):
        self.indir = indir
        self.number_file_cap = number_file_cap; self.number_cup = number_cup
        
        # Record all the file in folder, and put a cap on number of files 
        if isinstance(self.number_file_cap, int):
            file_list = [file for file in os.listdir(self.indir) if 'pkl' in file][:self.number_file_cap]
        else:
            file_list = self.number_file_cap
            
        file_list.sort()
        
        # Process
        if self.number_cup > 1:
            # Use mulithread processing
            with Pool(processes = self.number_cup) as pool:
                self.result = list(tqdm(pool.imap(self._process, file_list, 1), total = len(file_list)))
        
        else:
            # Use a for loop to process
            self.result = []
            with tqdm(total = len(file_list)) as pBar:
                for file in file_list:
                    pBar.update(1)
                    self.result.append(self._process(file))
        
        self.result, self.errors = zip(*self.result)
        
        # Concat result
        if len(self.result) !=0: self.result = pd.concat(self.result)
        
        return self.result
        
    def _process(self, file):
        
        self._process_list = [self._read_pickle,
                              self._reduce_events,
                              self._reduce_columns,
                             ]
        # initiate result in case of process failure
        temp = pd.DataFrame(); error = None
        rolling_kwarg = dict(file = file)
        
        for proc in self._process_list:
            try:
                rolling_kwarg = (proc(**rolling_kwarg))
            except Exception as error:
                return (temp, (proc.__name__,error))
            if rolling_kwarg['finish_flag']:
                return (rolling_kwarg['temp'], error)
            
    def _read_pickle(self, file):
        temp = pd.read_pickle(os.path.join(self.indir, file))
        return dict(temp = temp, file = file, finish_flag = False)

    def _reduce_events(self, temp, file, finish_flag):
        clist = ['CutS2Threshold', 'CutInteractionPeaksBiggest', 
                 '(z_3d_nn < 0)', 'CutDAQVeto', 'CutS1SingleScatter', 'CutFlash',
                 'CutBusyCheck', 'CutBusyTypeCheck', 'CutEndOfRunCheck', 
                ]
        self.clist = clist
        temp = temp[temp.eval('&'.join(clist))]
            
        return dict(temp = temp, file = file, finish_flag = False)

    def _reduce_columns(self, temp, file, finish_flag):
        clist = ['event_number', 'run_number', 'event_time',
                 'largest_other_s2', 's1_pattern_fit_bottom_hax', 's1_pattern_fit_hax',
                 's2_pattern_fit', 's2_area_fraction_top', 's2_range_50p_area', 's1_area_fraction_top',
                 's2_range_90p_area', 's2_range_80p_area',
                 'cs1', 'cs2', 's2', 's1', 'drift_time',   
                 'x', 'x_3d_nn', 'y', 'y_3d_nn', 'z', 'z_3d_nn',
                ] + list(set([c for c in temp.columns if 'Cut' in c]).difference(rnc.clist+['CutAllEnergy']))
        clist += [c for c in temp.columns if ('alt' in c) or ('largest_other' in c) or ('s2' in c)]
        clist = list(set(clist))
        temp = temp.loc[:, clist]
        return dict(temp = temp, file = file, finish_flag = True)

In [ ]:
import datetime, time
def spread_out_month(EFolder, n):
    flist = [f for f in os.listdir(EFolder) if 'pkl' in f]
    flist.sort()
    dates = np.array([datetime.datetime(2000+int(f[:2]), int(f[2:4]), 1) for f in flist])
    d = pd.DataFrame(dict(file=flist, date=dates))

    u, indices, counts = np.unique(dates, return_index=True, return_counts=True)
    mask = np.concatenate([indices[i]+np.random.choice(range(counts[i]), n) for i in range(len(u))])
    d = d.loc[mask,:]; print('Total sampling %d runs' %len(d))
    return d

In [ ]:
rnc = read_n_concat()
EFolder = '/project2/lgrandi/zhut/data/cleaned/pax_v6.8.0_Rn220_sciencerun1/'
d = spread_out_month(EFolder, 16)
time.sleep(1)
df_rn_1 = rnc.process(indir = EFolder, number_file_cap = d.file.values, number_cup = 1)

In [ ]:
rnc = read_n_concat()
EFolder = '/project2/lgrandi/zhut/data/compromise/pax_v6.8.0_none_sciencerun1_cpm/'
d = spread_out_month(EFolder, 16)
time.sleep(1)
df_bg_1 = rnc.process(indir = EFolder, number_file_cap = d.file.values, number_cup = 1)

In [ ]:
rnc = read_n_concat()
EFolder = '/project2/lgrandi/zhut/data/cleaned/pax_v6.8.0_AmBe_sciencerun1/'
d = spread_out_month(EFolder, 100)
time.sleep(1)
df_ab_1 = rnc.process(indir = EFolder, number_file_cap = d.file.values, number_cup = 1)

In [ ]:
df_rn_1 = df_rn_1[df_rn_1.CutFiducialCylinder1p3T]
df_ab_1 = df_ab_1[df_ab_1.CutFiducialZOptimized]
df_bg_1 = df_bg_1[df_bg_1.CutFiducialCylinder1p3T]
df_cb = pd.concat([df_rn_1, df_bg_1, df_ab_1])

In [ ]:
from sklearn.mixture import GaussianMixture
import pickle

gmix = GaussianMixture(n_components=2, covariance_type='full', max_iter=300, n_init=20)
mask = df_cb.eval('(largest_other_s2>0) & (s2>0) & (largest_other_s2_pattern_fit>0) \
       & ((largest_other_s2_delay_main_s1>6e3)|(largest_other_s2_delay_main_s1<0))')
trsa = df_cb[mask].sample(10000)
X = np.concatenate([np.log10(trsa.loc[:,['largest_other_s2', 'largest_other_s2_pattern_fit', 's2']]),
                    trsa.loc[:, ['event_time']]],
                   axis=1)
gmix = gmix.fit(X)

In [ ]:
with initiate_plot(20, 12):
    labels_ = gmix.predict(X)
    type0 = trsa[labels_==0]; type1 = trsa[labels_==1]
    
    plt.scatter(type0.largest_other_s2, type0.largest_other_s2_pattern_fit, edgecolor='none', s=10, color='darkred', alpha=0.3)
    plt.scatter(type1.largest_other_s2, type1.largest_other_s2_pattern_fit, edgecolor='none', s=10, color='midnightblue', alpha=0.3)

    plt_config(xlim=[10, 1e7], ylim=[10, 1e6], xlabel='Largest Other S2 [pe]', ylabel='Largest Other S2 Pattern Likelihood',
              title='Nonparametric Clustering Input Samples')
    plt.xscale('log'); plt.yscale('log');
    fig.savefig('dbd_s2single_clusteing_input.png', bbox_inches='tight')

In [ ]:
with open('/project2/lgrandi/zhut/s2_single_classifier_gmix.pkl', 'rb') as f:
    gmix=pickle.load(f)

In [ ]:
#with open('/project2/lgrandi/zhut/s2_single_classifier_gmix.pkl', 'wb') as f:
#    pickle.dump(gmix, f)

In [ ]:
def classify(df):
    df['class_number'] = 0
    
    test = gmix.predict([[1e5, 4e3, 3e5, np.mean(df_cb.event_time)]])[0]
    
    mask = df.eval('(largest_other_s2>0) & (s2>0) & (largest_other_s2_pattern_fit>0) \
    & ((largest_other_s2_delay_main_s1>6e3)|(largest_other_s2_delay_main_s1<0))')
    Y = np.concatenate([np.log10(df.loc[mask,['largest_other_s2', 'largest_other_s2_pattern_fit', 's2']]),
                    df.loc[mask, ['event_time']]],
                   axis=1)
    df.loc[mask,'class_number'] = np.abs(gmix.predict(Y) - (1-test))
    return df

df_cb = classify(df_cb)

In [ ]:
def s2_single_scatter(x):
    y0, y1 = x*0.00832+72.3, x*0.03-109
    return y0/(np.exp((x-23300)*5.91e-4)+1)+y1/(np.exp((23300-x)*5.91e-4)+1)

In [ ]:
from matplotlib.colors import ListedColormap
def plotting_the_rest(right=False):
    cmap = plt.get_cmap('autumn_r')
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:,-1] = np.linspace(0, 0.7, cmap.N)
    Omy_cmap = ListedColormap(my_cmap)
    
    cmap = plt.get_cmap('winter_r')
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:,-1] = np.linspace(0, 0.7, cmap.N)
    Bmy_cmap = ListedColormap(my_cmap)
    
    plt.pcolormesh(x_bin_edges, y_bin_edges, nh_a.T, norm=LogNorm(), cmap=plt.get_cmap('viridis'), alpha=.05)
    plt.pcolormesh(x_bin_edges, y_bin_edges, nh_0.T, norm=LogNorm(), cmap=Omy_cmap)
    plt.pcolormesh(x_bin_edges, y_bin_edges, nh_1.T, norm=LogNorm(), cmap=Bmy_cmap)
    
    xgrid = x_bin_edges
    
    
    if right:
        plt.plot(xgrid, s2_single_scatter(xgrid), 'red', ls='--')
        plt_config(xlim=[x_bin_edges[0], x_bin_edges[-1]], ylim=[y_bin_edges[0], y_bin_edges[-1]],
              xlabel='S2 [pe]', ylabel='Largest Other S2 [pe]', title='Clustering Result')
        return None
    plt.plot(xgrid, 0.1064*xgrid*0.65 + 758.80*(xgrid*0.65)**0.05639 - 819.29, 'red', ls='--')
    plt_config(xlim=[x_bin_edges[0], x_bin_edges[-1]], ylim=[y_bin_edges[0], y_bin_edges[-1]],
              xlabel='Largest Other S2 [pe]', ylabel='Largest Other S2 Pattern Likelihood',
              title='Clustering Result Where %d<S2<%d' %(dep/10**0.5, dep*10**0.5))
    
dfs = ['df_cb_0', 'df_cb_1', 'df_cb_all', 'df_cb']
for jx, dep in enumerate(np.logspace(2.5, 6.5, 5)[:]):
    ########
    with initiate_plot(20, 12):
        argx, argy = '{df}.largest_other_s2', '{df}.largest_other_s2_pattern_fit'
        x_bin_edges, y_bin_edges = np.logspace(1,6.6,151), np.logspace(1,6.0,151)
        
        df_cb_all = df_cb[(np.abs(np.log10(df_cb.s2/dep)) < 0.5)]
        nh_a,_,_ = np.histogram2d(eval(argx.format(df=dfs[3])), eval(argy.format(df=dfs[3])), bins=[x_bin_edges, y_bin_edges])
        
        df_cb_0 = df_cb_all[(df_cb_all.class_number==0)]
        nh_0,_,_ = np.histogram2d(eval(argx.format(df=dfs[0])), eval(argy.format(df=dfs[0])), bins=[x_bin_edges, y_bin_edges])
        df_cb_1 = df_cb_all[(df_cb_all.class_number==1)]
        nh_1,_,_ = np.histogram2d(eval(argx.format(df=dfs[1])), eval(argy.format(df=dfs[1])), bins=[x_bin_edges, y_bin_edges])
        
        ax = fig.add_subplot(111); plotting_the_rest()
        plt.xscale('log'); plt.yscale('log')
        fig.savefig('dbd_s2single_energy_series_%d.png'%jx, bbox_inches='tight')

In [ ]:
with initiate_plot(20, 12):       
    ####
    argx, argy = '{df}.s2', '{df}.largest_other_s2'
    x_bin_edges, y_bin_edges = np.logspace(2.1,6.8,151), np.logspace(1,6.8,151)

    df_cb_all = df_cb #[(np.abs(np.log10(df_cb.s2/dep)) < 0.5)]
    nh_a,_,_ = np.histogram2d(eval(argx.format(df=dfs[3])), eval(argy.format(df=dfs[3])), bins=[x_bin_edges, y_bin_edges])

    df_cb_0 = df_cb_all[(df_cb_all.class_number==0)]
    nh_0,_,_ = np.histogram2d(eval(argx.format(df=dfs[0])), eval(argy.format(df=dfs[0])), bins=[x_bin_edges, y_bin_edges])
    df_cb_1 = df_cb_all[(df_cb_all.class_number==1)]
    nh_1,_,_ = np.histogram2d(eval(argx.format(df=dfs[1])), eval(argy.format(df=dfs[1])), bins=[x_bin_edges, y_bin_edges])

    ax = fig.add_subplot(111); plotting_the_rest(right=True)
    plt.xscale('log'); plt.yscale('log')
    fig.savefig('dbd_s2single_original_space.png', bbox_inches='tight')

In [ ]:
splits = [int(datetime.datetime(2017,m,1,0,0).strftime('%s'))*1e9 for m in [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
splits += [int(datetime.datetime(2018,m,1,0,0).strftime('%s'))*1e9 for m in [1, 2]]
df_cb['month_from_start'] = np.digitize(df_cb.event_time, splits)

In [ ]:
from matplotlib.colors import ListedColormap
def plotting_the_rest(right=False):
    cmap = plt.get_cmap('autumn_r')
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:,-1] = np.linspace(0, 0.7, cmap.N)
    Omy_cmap = ListedColormap(my_cmap)
    
    cmap = plt.get_cmap('winter_r')
    my_cmap = cmap(np.arange(cmap.N))
    my_cmap[:,-1] = np.linspace(0, 0.7, cmap.N)
    Bmy_cmap = ListedColormap(my_cmap)
    
    plt.pcolormesh(x_bin_edges, y_bin_edges, nh_a.T, norm=LogNorm(), cmap=plt.get_cmap('viridis'), alpha=.05)
    plt.pcolormesh(x_bin_edges, y_bin_edges, nh_0.T, norm=LogNorm(), cmap=Omy_cmap)
    plt.pcolormesh(x_bin_edges, y_bin_edges, nh_1.T, norm=LogNorm(), cmap=Bmy_cmap)
    
    xgrid = x_bin_edges
    
    
    if right:
        plt.plot(xgrid, s2_single_scatter(xgrid), 'red', ls='--')
        plt_config(xlim=[x_bin_edges[0], x_bin_edges[-1]], ylim=[y_bin_edges[0], y_bin_edges[-1]],
              xlabel='S2 [pe]', ylabel='Largest Other S2 [pe]', title='Clustering Result')
        return None
    plt.plot(xgrid, 0.1064*xgrid*0.65 + 758.80*(xgrid*0.65)**0.05639 - 819.29, 'red', ls='--')
    plt_config(xlim=[x_bin_edges[0], x_bin_edges[-1]], ylim=[y_bin_edges[0], y_bin_edges[-1]],
              xlabel='Largest Other S2 [pe]', ylabel='Largest Other S2 Pattern Likelihood',
              title='Clustering Result During %s' % ms[jx]
              )

dfs = ['df_cb_0', 'df_cb_1', 'df_cb_all', 'df_cb']
monthes = df_cb.month_from_start.unique()
monthes.sort()
ms = ['Feb 2017', 'Mar 2017', 'Apr 2017', 'May 2017', 'Jun 2017', 'Jul 2017', 
      'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018']
for jx, mo in enumerate(monthes):
    ########
    with initiate_plot(20, 12):
        argx, argy = '{df}.largest_other_s2', '{df}.largest_other_s2_pattern_fit'
        x_bin_edges, y_bin_edges = np.logspace(1,6.6,151), np.logspace(1,6.0,151)
        
        df_cb_all = df_cb[df_cb.month_from_start==mo]
        nh_a,_,_ = np.histogram2d(eval(argx.format(df=dfs[3])), eval(argy.format(df=dfs[3])), bins=[x_bin_edges, y_bin_edges])
        
        df_cb_0 = df_cb_all[(df_cb_all.class_number==0)]
        nh_0,_,_ = np.histogram2d(eval(argx.format(df=dfs[0])), eval(argy.format(df=dfs[0])), bins=[x_bin_edges, y_bin_edges])
        df_cb_1 = df_cb_all[(df_cb_all.class_number==1)]
        nh_1,_,_ = np.histogram2d(eval(argx.format(df=dfs[1])), eval(argy.format(df=dfs[1])), bins=[x_bin_edges, y_bin_edges])
        
        ax = fig.add_subplot(111); plotting_the_rest()
        plt.xscale('log'); plt.yscale('log')
        fig.savefig('dbd_s2single_time_series_%d.png'%jx, bbox_inches='tight')